<a href="https://colab.research.google.com/github/ImVikashKr/Translate/blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q openai

In [ ]:
# Import necessary libraries
import pandas as pd  #Pandas for Data Manipulation
import openai        #Open AI for AI!
import numpy as np
import math
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/items.csv')

In [ ]:
data.head()

,Unnamed: 0,id,account_id,parent_id,owner_id,owner_type,name,hash,url,metadata,...,type,subtype,extension,mimetype,created_by,source_id,status,is_deleted,updated,created
0,0,002f4afa72b82a502223c72beda113285ec1bbe9,9,NaN,1199,case,Notice Format.docx,03051e5a6e568fda637cf01f8e398f8912822d431b19fa...,https://artemis-documents-v1.s3.ap-south-1.ama...,"{""size"": 25940}",...,document,docx,docx,application/vnd.openxmlformats-officedocument....,6469545973,NaN,1,0,2022-08-04 08:10:45,2022-08-04 08:10:45
1,1,009d455a6c2d4bbcb5ad0ad1f05164d2437e25ba,8,NaN,4193347700,user-profile,undefined,NaN,NaN,"{""size"": null}",...,NaN,NaN,NaN,NaN,6911322259,NaN,1,0,2022-08-14 18:32:05,2022-08-14 18:32:05
2,2,00eae01b385323d4a1d76c5510049f6c8ccc95c9,10,NaN,1019,case,apple-touch-icon.png,548e12b2a3a3efd0aa9f8b15009002d90485db9b47393e...,https://artemis-documents-v1.s3.ap-south-1.ama...,"{""size"": 25701}",...,image,png,png,image/png,52697158,NaN,1,0,2022-07-05 10:27:29,2022-07-05 10:27:29
3,3,0128d3a471c4243e70e21a76fc02ca744eeada11,1,NaN,1306413353,user-profile,NaN,NaN,NaN,"{""size"": null}",...,NaN,NaN,NaN,NaN,1306413353,NaN,1,0,2021-06-10 09:02:52,2021-06-10 09:02:52
4,4,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,9,NaN,1957,case,Applicant user guide.pdf,de97e6380dbc9351fa8ec3e380a5adec3dc7ada4849c13...,https://artemis-documents-v1.s3.ap-south-1.ama...,"{""size"": 13836839}",...,document,pdf,pdf,application/pdf,2835371597,NaN,1,0,2022-08-06 09:38:34,2022-08-06 09:38:34


In [ ]:
for i in data['url']:
  print(i)

https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/03051e5a6e568fda637cf01f8e398f8912822d431b19fa5cd8605bdc67a4291d.docx
nan
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/548e12b2a3a3efd0aa9f8b15009002d90485db9b47393ec35fa4efec1b9edb65.png
nan
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/de97e6380dbc9351fa8ec3e380a5adec3dc7ada4849c13d2d1d707bb42a1e282.pdf
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/ce0d54fd900af894d9385f7556721bd472e7d0dccfe845cf48f6a8cb0188cc4b.pdf
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/840843d212908e9571aba6fd3b7a772454a370b52264dfa06d6226e0360acad0.docx
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/4b203076da3ae12bd918ae8c8419ff0d7b7e31c6a267d8122c7046afb7b6bd86.docx
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/d55a59ac7a6fa0360cf83a3bfc60681e16f75a10bb9ef04b0e22d7377faf24ee.pdf
https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/85e60b2552fdf87313b208f6ca57dc69c79a9f95ff75d4939b

In [ ]:
data.columns

Index(['Unnamed: 0', 'id', 'account_id', 'parent_id', 'owner_id', 'owner_type',
       'name', 'hash', 'url', 'metadata', 'service_provider_metadata', 'type',
       'subtype', 'extension', 'mimetype', 'created_by', 'source_id', 'status',
       'is_deleted', 'updated', 'created'],
      dtype='object')

In [ ]:
!pip install -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -q python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q docx2txt

  Preparing metadata (setup.py) ... done


In [ ]:
!sudo apt-get install -q antiword

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  antiword
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 118 kB of archives.
After this operation, 603 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 antiword amd64 0.37-16 [118 kB]
Fetched 118 kB in 0s (259 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package antiword.
(Reading database ... 122518 files and directories currently installed.)
P

In [ ]:
data['type'].value_counts()

document     1282
image         575
file          262
folder         29
video          29
animation       1
Name: type, dtype: int64

In [ ]:
# Filter the filenames that end with .pdf, .docx, and .doc and assign them to a new column pdf_docx
pdf_mask = data['url'].str.endswith(('.pdf'))
pdf_mask = pdf_mask.fillna(False)  # replace NaN with False
data['pdf'] = data['url'][pdf_mask]

In [ ]:
docx_mask = data['url'].str.endswith(('.docx'))
docx_mask = docx_mask.fillna(False)  # replace NaN with False
data['docx'] = data['url'][docx_mask]

In [ ]:
doc_mask = data['url'].str.endswith(('.doc'))
doc_mask = doc_mask.fillna(False)  # replace NaN with False
data['doc'] = data['url'][doc_mask]

In [ ]:
pdf_df = data[['id', 'pdf']]

In [ ]:
pdf_df.dropna(subset=['pdf'], inplace=True)

<ipython-input-17-a030597e3cca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_df.dropna(subset=['pdf'], inplace=True)


In [ ]:
docx_df = data[['id', 'docx']]

In [ ]:
docx_df.dropna(subset=['docx'], inplace=True)

<ipython-input-19-4a50c6a8e757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docx_df.dropna(subset=['docx'], inplace=True)


In [ ]:
doc_df = data[['id', 'doc']]

In [ ]:
doc_df.dropna(subset=['doc'], inplace=True)

<ipython-input-21-fe68ba2690d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df.dropna(subset=['doc'], inplace=True)


In [ ]:
pdf_df

,id,pdf
4,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...
5,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...
8,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...
11,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...
12,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...
...,...,...
2333,ff74866fa68286580602b51b100d2f84db3c8fcd,https://artemis-documents-v1.s3.ap-south-1.ama...
2334,ff792a5cc96b6019590cd19657de41a99009d707,https://artemis-documents-v1.s3.ap-south-1.ama...
2336,ff90a3673797177d3e9c79a5a21e299ed773a2bc,https://artemis-documents-v1.s3.ap-south-1.ama...
2337,ff918298e3bcf1cce14388227ee86442f31a51f4,https://artemis-documents-v1.s3.ap-south-1.ama...


In [ ]:
docx_df

,id,docx
0,002f4afa72b82a502223c72beda113285ec1bbe9,https://artemis-documents-v1.s3.ap-south-1.ama...
6,0164b4535d51211f8b0329f345b24b2f49b35230,https://artemis-documents-v1.s3.ap-south-1.ama...
7,016670df7daa050d4f27ecff5b329d1deb13cb16,https://artemis-documents-v1.s3.ap-south-1.ama...
14,023260deaec35362671abbfca0f7a18314e71c51,https://artemis-documents-v1.s3.ap-south-1.ama...
15,02380bbdd3274ad8b7b5a89a63e469d70e6d01f2,https://artemis-documents-v1.s3.amazonaws.com/...
...,...,...
2275,fa0f6a0f812c9f14913b32d724617e248d049446,https://artemis-documents-v1.s3.ap-south-1.ama...
2290,fbd0a885f571553f60824b4a4cdc528681353ad6,https://artemis-documents-v1.s3.ap-south-1.ama...
2301,fd3bccc6796c860dd784c42bf4428631f4fe5fa0,https://artemis-documents-v1.s3.ap-south-1.ama...
2311,fdad2aa00b833aeabe9129b9165b917d12158cfc,https://artemis-documents-v1.s3.ap-south-1.ama...


In [ ]:
doc_df

,id,doc
26,032d05890bb2124d7dd7db1ba61b1c8b835ca56a,https://artemis-documents-v1.s3.ap-south-1.ama...
101,0ac02705cfcedd8ba12fa4da545e04b9e649f268,https://artemis-documents-v1.s3.ap-south-1.ama...
116,0c6f24e44d30269bb5a17572c149afbb14700fed,https://artemis-documents-v1.s3.ap-south-1.ama...
124,0d812e71834c2a4b33665998df69605b357d42a4,https://artemis-documents-v1.s3.amazonaws.com/...
201,169268af0234a5f144a1e6067bd39ac8633846cf,https://artemis-documents-v1.s3.ap-south-1.ama...
...,...,...
2210,f45742e2a083f61b06954fa29760526229402657,https://artemis-documents-v1.s3.ap-south-1.ama...
2216,f4d4f8459536a2ea09094e99862f97ec9760dbc9,https://artemis-documents-v1.s3.ap-south-1.ama...
2231,f657ac46b4320a7e732face58cb259c7175b32d1,https://artemis-documents-v1.s3.ap-south-1.ama...
2293,fc447ea01071861a78c35dc7c9a3496bc6affd67,https://artemis-documents-v1.s3.ap-south-1.ama...


In [ ]:
import io
import requests
from PyPDF2 import PdfReader

def add_pdf_text_column(df, pdf_column_name, pdf_text_column_name):
    """
    Adds a column to a DataFrame with the text content of the PDFs 
    referenced in the specified column of the DataFrame.

    :param df: The DataFrame to add the column to.
    :param pdf_column_name: The name of the column containing the PDF URLs.
    :param pdf_text_column_name: The name to give the new column containing the PDF text.
    :return: The modified DataFrame.
    """
    texts = []
    for pdf_url in df[pdf_column_name]:
        try:
            response = requests.get(pdf_url)
            response.raise_for_status()

            with io.BytesIO(response.content) as f:
                reader = PdfReader(f)
                text = ''
                for page in reader.pages:
                    text += page.extract_text()

                texts.append(text)
        except:
            texts.append('')

    df[pdf_text_column_name] = texts
    return df


In [ ]:
pdf_df = add_pdf_text_column(pdf_df, 'pdf', 'pdf_text')

In [ ]:
pdf_df.head()

In [ ]:
interaction_filename = "pdf_df.csv"
pdf_df.to_csv("/content/drive/MyDrive/"+interaction_filename, index=False, float_format='%.0f', escapechar='\\')


In [ ]:
import io
import requests
import docx
import pandas as pd


def extract_docx_text(data: pd.DataFrame, url_column: str, text_column: str) -> pd.DataFrame:
    """
    Extract text from Docx URLs in a given column and store in a new column.
    
    Args:
    - data: Pandas DataFrame containing the data.
    - url_column: Name of the column containing Docx URLs.
    - text_column: Name of the column where extracted text will be stored.
    
    Returns:
    - Pandas DataFrame with the extracted text stored in a new column.
    """
    # Iterate over the rows of the DataFrame
    for index, row in data.iterrows():
        # Get the Docx URL from the specified column
        url = row[url_column]
        # Download the Docx file
        response = requests.get(url)
        # Create an in-memory file-like object from the downloaded content
        f = io.BytesIO(response.content)
        try:
            # Create a Document object from the in-memory file-like object
            doc = docx.Document(f)
        except Exception as e:
            # Print any exception that may have occurred while trying to open the Docx file
            print(f"Error occurred while opening Docx file from URL {url}: {e}")
            continue
        # Extract the text from the Document object
        text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        # Store the extracted text in a new column
        data.at[index, text_column] = text
        
    return data


In [ ]:
docx_df = extract_docx_text(docx_df, 'docx', 'docx_text')

In [ ]:
docx_df

In [ ]:
interaction = "docx_df.csv"
docx_df.to_csv("/content/drive/MyDrive/"+interaction, index=False, float_format='%.0f', escapechar='\\')

In [ ]:
import io
import requests
import subprocess

def add_doc_text_column(df, doc_column_name, doc_text_column_name):
    """
    Adds a column to a DataFrame with the text content of the .doc files
    referenced in the specified column of the DataFrame.

    :param df: The DataFrame to add the column to.
    :param doc_column_name: The name of the column containing the .doc URLs.
    :param doc_text_column_name: The name to give the new column containing the .doc text.
    :return: The modified DataFrame.
    """
    for i, doc_url in enumerate(df[doc_column_name]):
        try:
            response = requests.get(doc_url)
            response.raise_for_status()

            with io.BytesIO(response.content) as f:
                output = subprocess.check_output(["antiword", "-"], input=f.read())
                text = output.decode("utf-8")

                df.at[i, doc_text_column_name] = text
        except:
            df.at[i, doc_text_column_name] = ''

    return df


In [ ]:
doc_df = add_doc_text_column(doc_df, 'doc', 'doc_text')

In [ ]:
doc_df

In [ ]:
interaction = "doc_df.csv"
doc_df.to_csv("/content/drive/MyDrive/"+interaction, index=False, float_format='%.0f', escapechar='\\')

In [ ]:
pdf_df

In [ ]:
pdf_df['pdf_text']

In [ ]:
pdf_df = pd.read_csv('/content/drive/MyDrive/pdf_df.csv')

In [ ]:
docx_df = pd.read_csv('/content/drive/MyDrive/docx_df.csv')

In [ ]:
doc_df = pd.read_csv('/content/drive/MyDrive/doc_df.csv')

In [ ]:
pdf_df['pdf_text'] = pdf_df.pdf_text.replace("\n", " ")

In [ ]:
pdf_df.head()

,id,pdf,pdf_text
0,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...,RSLS AJUPITICE L OK AD AL A T PL A TFORM \nApp...
1,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...,S. No. Name Hosting By Performance Department\...
2,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...,Test File\n
3,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...,Online Payment\nAdd Administration Fee\nRemo...
4,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...,MARRIAGE \nCERTIFICATE\nTHIS IS TO CERTIFY THA...


In [ ]:
docx_df['docx_text'] = docx_df.docx_text.replace("\n", " ")

In [ ]:
docx_df.head()

,id,docx,docx_text
0,002f4afa72b82a502223c72beda113285ec1bbe9,https://artemis-documents-v1.s3.ap-south-1.ama...,\n………………. District Legal Services Authority \n...
1,0164b4535d51211f8b0329f345b24b2f49b35230,https://artemis-documents-v1.s3.ap-south-1.ama...,NaN
2,016670df7daa050d4f27ecff5b329d1deb13cb16,https://artemis-documents-v1.s3.ap-south-1.ama...,\nIn consistence with data protection guidelin...
3,023260deaec35362671abbfca0f7a18314e71c51,https://artemis-documents-v1.s3.ap-south-1.ama...,kjbcbscklcblsbncljscljsabcljnscljs c
4,02380bbdd3274ad8b7b5a89a63e469d70e6d01f2,https://artemis-documents-v1.s3.amazonaws.com/...,RENT AGREEMENT\nThis Rent agreement is hereby ...


In [ ]:
doc_df['doc_text'] = doc_df.doc_text.replace("\n", " ")

In [ ]:
pdf_df.dropna(subset=['pdf_text'], inplace=True)
docx_df.dropna(subset=['docx_text'], inplace=True)
doc_df.dropna(subset=['doc_text'], inplace=True)

#model 
https://www.sbert.net/docs/pretrained_models.html

In [ ]:
doc_df.head()

,id,doc,doc_text
0,032d05890bb2124d7dd7db1ba61b1c8b835ca56a,https://artemis-documents-v1.s3.ap-south-1.ama...,\n\n|LRA Form 7.13 |REQUEST FOR ARBIT...
69,NaN,NaN,\n| | ...
70,NaN,NaN,\n 调解申请书\n ...
71,NaN,NaN,\n| | ...
72,NaN,NaN,\n SETTLEMENT AGREE...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
import numpy as np

# Install necessary NLTK models and resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from gensim.models import KeyedVectors

# Load pre-trained word embedding model
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/sampleWebsite/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# Define function for cleaning and preprocessing text
def clean_text(text):
    # Remove unwanted characters
    text = text.replace('\n', ' ')
    text = text.replace('\r', '')
    text = text.replace('\t', ' ')
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

In [ ]:
import re
import nltk
nltk.download('punkt')

pdf_df['pdf_text'] = pdf_df['pdf_text'].replace(np.nan, '', regex=True)

# Clean and preprocess the 'pdf_text' column
pdf_df['clean_text'] = pdf_df['pdf_text'].apply(lambda x: clean_text(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
pdf_df

,id,pdf,pdf_text,clean_text
0,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...,RSLS AJUPITICE L OK AD AL A T PL A TFORM \nApp...,"[RSLS, AJUPITICE, L, OK, AD, AL, A, T, PL, A, ..."
1,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...,S. No. Name Hosting By Performance Department\...,"[S, No, Name, Hosting, By, Performance, Depart..."
2,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...,Test File\n,"[Test, File]"
3,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...,Online Payment\nAdd Administration Fee\nRemo...,"[Online, Payment, Add, Administration, Fee, Re..."
4,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...,MARRIAGE \nCERTIFICATE\nTHIS IS TO CERTIFY THA...,"[MARRIAGE, CERTIFICATE, THIS, IS, TO, CERTIFY,..."
...,...,...,...,...
922,ff74866fa68286580602b51b100d2f84db3c8fcd,https://artemis-documents-v1.s3.ap-south-1.ama...,A Simple PDF File \n This is a small demonstr...,"[A, Simple, PDF, File, This, small, demonstrat..."
923,ff792a5cc96b6019590cd19657de41a99009d707,https://artemis-documents-v1.s3.ap-south-1.ama...,From :\nno-r eply@app.jupitice.com\nTo : godaw...,"[From, eplyappjupiticecom, To, godawarifarmsya..."
924,ff90a3673797177d3e9c79a5a21e299ed773a2bc,https://artemis-documents-v1.s3.ap-south-1.ama...,1 \n COMMONWEALTH OF THE BAHAMAS \nIN THE SUP...,"[1, COMMONWEALTH, OF, THE, BAHAMAS, IN, THE, S..."
925,ff918298e3bcf1cce14388227ee86442f31a51f4,https://artemis-documents-v1.s3.ap-south-1.ama...,Terms of Reference \nSenior Consultant (Legal...,"[Terms, Reference, Senior, Consultant, Legal, ..."


In [ ]:
embeddings = []
for words in pdf_df['clean_text']:
    embeddings.append([model[word] for word in words if word in model.key_to_index.keys()])

# Aggregate the word embeddings for each row in the 'pdf_df' DataFrame
pdf_df['embedding'] = pd.Series(embeddings).apply(lambda x: np.mean(x, axis=0))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
pdf_df['embedding']

0      [-0.04796342, 0.09589727, -0.002151811, 0.0670...
1      [-0.012620071, 0.015516347, 0.0650911, 0.04459...
2      [0.08496094, 0.052734375, -0.09753418, 0.11022...
3      [9.2726485e-05, 0.10807918, -0.03481273, 0.050...
4      [0.008904593, -0.13824463, 0.18371582, 0.19172...
                             ...                        
922                                                  NaN
923                                                  NaN
924                                                  NaN
925                                                  NaN
926                                                  NaN
Name: embedding, Length: 862, dtype: object

In [ ]:
pdf_df['embedding'] = pdf_df['embedding'].apply(lambda x: np.mean(x, axis=0) if isinstance(x, np.ndarray) and len(x) > 0 else np.zeros(300))


In [ ]:
pdf_df

,id,pdf,pdf_text,clean_text,embedding
0,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...,RSLS AJUPITICE L OK AD AL A T PL A TFORM \nApp...,"[RSLS, AJUPITICE, L, OK, AD, AL, A, T, PL, A, ...",-0.003808
1,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...,S. No. Name Hosting By Performance Department\...,"[S, No, Name, Hosting, By, Performance, Depart...",-0.010966
2,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...,Test File\n,"[Test, File]",0.000651
3,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...,Online Payment\nAdd Administration Fee\nRemo...,"[Online, Payment, Add, Administration, Fee, Re...",-0.004814
4,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...,MARRIAGE \nCERTIFICATE\nTHIS IS TO CERTIFY THA...,"[MARRIAGE, CERTIFICATE, THIS, IS, TO, CERTIFY,...",-0.002815
...,...,...,...,...,...
922,ff74866fa68286580602b51b100d2f84db3c8fcd,https://artemis-documents-v1.s3.ap-south-1.ama...,A Simple PDF File \n This is a small demonstr...,"[A, Simple, PDF, File, This, small, demonstrat...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
923,ff792a5cc96b6019590cd19657de41a99009d707,https://artemis-documents-v1.s3.ap-south-1.ama...,From :\nno-r eply@app.jupitice.com\nTo : godaw...,"[From, eplyappjupiticecom, To, godawarifarmsya...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
924,ff90a3673797177d3e9c79a5a21e299ed773a2bc,https://artemis-documents-v1.s3.ap-south-1.ama...,1 \n COMMONWEALTH OF THE BAHAMAS \nIN THE SUP...,"[1, COMMONWEALTH, OF, THE, BAHAMAS, IN, THE, S...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
925,ff918298e3bcf1cce14388227ee86442f31a51f4,https://artemis-documents-v1.s3.ap-south-1.ama...,Terms of Reference \nSenior Consultant (Legal...,"[Terms, Reference, Senior, Consultant, Legal, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
from sklearn.metrics.pairwise import cosine_distances

# Define a function to calculate the distances between two embeddings
def distance_between_embeddings(embedding1, embedding2):
    return cosine_distances(embedding1.reshape(1,-1), embedding2.reshape(1,-1))[0][0]

# Get user input
query = input("Enter your PDF recommendation query: ")

Enter your PDF recommendation query: crime repot


In [ ]:
# Convert query to embedding
def get_embedding(text):
    # Split text into words and remove punctuation
    words = [word.strip('.,!?"\'').lower() for word in text.split()]
    # Filter out words that are not in the vocabulary of the model
    words = [word for word in words if word in model.key_to_index.keys()]
    # Compute the mean of the word embeddings
    if len(words) == 0:
        # Return a zero vector if no words are in the vocabulary
        return np.zeros(model.vector_size)
    else:
        return np.mean([model[word] for word in words], axis=0)

query_embedding = get_embedding(query)

In [ ]:
# Calculate the distances between the query embedding and the PDF embeddings using the cosine distance metric
distances = [distance_between_embeddings(query_embedding, embedding) for embedding in pdf_df['embedding']]

# Get the indices of the nearest neighbors (i.e., the PDFs with the smallest distances)
indices_of_nearest_neighbors = np.argsort(distances)

# Print the titles of the top 5 recommended PDFs
print("Titles of top 5 recommended PDFs:")
print(pdf_df.loc[indices_of_nearest_neighbors[:5]]['title'])

# Print the links of the top 5 recommended PDFs
print("\nLinks of top 5 recommended PDFs:")
print(pdf_df.loc[indices_of_nearest_neighbors[:5]]['pdf'])


ValueError: ignored